In [1]:
#imports
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from pathlib import Path
import seaborn as sns
import time

#tensorflow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


#keras
import keras
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import Adam

#image stuff
from PIL import Image, ImageOps

#sklearn
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics

from scipy.ndimage import gaussian_filter

2021-12-06 17:03:53.882175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-06 17:03:53.882221: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Data Processing

In [ ]:
# importing and resizing image data 
# change path to wherever your data lives on your computer
csv_path = "/home/jovyan/w207-final-project/data/train.csv"
image_path = '/home/jovyan/w207-final-project/data/train/'

def CutenessBuckets(pawpularity_score):
    if pawpularity_score <= 10:
        return 0
    elif pawpularity_score <= 20:
        return 1
    elif pawpularity_score <= 30:
        return 2
    elif pawpularity_score <= 40:
        return 3
    elif pawpularity_score <= 50:
        return 4
    elif pawpularity_score <= 60:
        return 5
    elif pawpularity_score <= 70:
        return 6
    elif pawpularity_score <= 80:
        return 7
    elif pawpularity_score <= 90:
        return 8
    elif pawpularity_score <= 100:
        return 9

def CleanandProcessData(csv_path, image_path, image_resize):

    # imnporting csv with image ids and pawpularity scores
    df = pd.read_csv(csv_path)

    # creating path to image using the id in the data frame we've created
    df['path'] = image_path + df['Id'] + '.jpg'
    df['img'] = df['path'].apply(lambda x: Image.open(x).convert("RGB").resize((image_resize,image_resize)))
    
    df['cuteness_bin'] = df['Pawpularity'].apply(CutenessBuckets)
    df['img_array'] = df['img'].apply(lambda x: img_to_array(x)/255)

    labels_raw=[]
    labels_bin = []
    imgArr=[]

    for ind in df.index:
        img = df.img_array[0]
        imgArr.append(img)
        labels_raw.append(df['Pawpularity'][ind])
        labels_bin.append(df['cuteness_bin'][ind])

    labels_raw = np.array(labels_raw)
    labels_bin = np.array(labels_bin)
    X_train=np.array(imgArr, dtype=float)
    print('Training Shape:',X_train.shape)
    print('Labels Shape;', labels_raw.shape)
    return X_train, labels_raw, labels_bin

X_train, labels_raw, labels_bin = CleanandProcessData(csv_path, image_path, 20)

In [ ]:
# Shuffling data and separating into train, test and dev sets
shuffle = np.random.permutation(np.arange(X_train.shape[0]))

X_train, labels_raw, labels_bin = X_train[shuffle], labels_raw[shuffle], labels_bin[shuffle]

print('data shape: ', X_train.shape)
print('pawpularity label shape:', labels_raw.shape)
print('bin label shape:', labels_bin.shape)

# Set some variables to hold test, dev, and training data.
train_data, train_labels_raw, train_labels_bin = X_train[:7921], labels_raw[:7921], labels_bin[:7921]
test_data, test_labels_raw, test_labels_bin = X_train[8916:], labels_raw[8916:], labels_bin[8916:]
dev_data, dev_labels_raw, dev_labels_bin = X_train[7920:8916], labels_raw[7920:8916], labels_bin[7920:8916]

# raw labels (ie original pawpularity score)
test_labels_category = to_categorical(test_labels_raw)
train_labels_category = to_categorical(train_labels_raw)
dev_labels_category = to_categorical(dev_labels_raw)

# bucketed labels 
test_labels_bins_category = to_categorical(test_labels_bin)
train_labels_bins_category = to_categorical(train_labels_bin)
dev_labels_bins_category = to_categorical(dev_labels_bin)

# blurred image data (optional use)
blurred_train = gaussian_filter(train_data, sigma=1)
blurred_test = gaussian_filter(test_data, sigma = 1)

In [ ]:
# Classical ML models
# have to reshape the data to input it into a different model
train_size = train_data.shape[0]
train_shape = train_data.shape[1]*train_data.shape[2]*train_data.shape[3]

dev_size = dev_data.shape[0]
dev_shape = dev_data.shape[1]*dev_data.shape[2]*dev_data.shape[3]

test_size = test_data.shape[0]
test_shape = test_data.shape[1]*test_data.shape[2]*test_data.shape[3]

train_data_temp = train_data.reshape(train_size,train_shape)
dev_data_temp = dev_data.reshape(dev_size,dev_shape)
test_data_temp = test_data.reshape(test_size,test_shape)

In [ ]:
#buckets
def CutenessBuckets_2(pawpularity_score):
    if pawpularity_score <= 50:
        return 0
    elif pawpularity_score <= 100:
        return 1

    
def CutenessBuckets_3(pawpularity_score):
    if pawpularity_score <= 33:
        return 0
    elif pawpularity_score <= 66:
        return 1
    elif pawpularity_score <= 100:
        return 2

def CutenessBuckets_4(pawpularity_score):
    if pawpularity_score <= 25:
        return 0
    elif pawpularity_score <= 50:
        return 1
    elif pawpularity_score <= 75:
        return 2
    elif pawpularity_score <= 100:
        return 3
    
def CutenessBuckets_5(pawpularity_score):
    if pawpularity_score <= 20:
        return 0
    elif pawpularity_score <= 40:
        return 1
    elif pawpularity_score <= 60:
        return 2
    elif pawpularity_score <= 80:
        return 3
    elif pawpularity_score <= 100:
        return 4
    
def CutenessBuckets_7(pawpularity_score):
    if pawpularity_score <= 15:
        return 0
    elif pawpularity_score <= 30:
        return 1
    elif pawpularity_score <= 45:
        return 2
    elif pawpularity_score <= 60:
        return 3
    elif pawpularity_score <= 75:
        return 4
    elif pawpularity_score <= 90:
        return 5
    elif pawpularity_score <= 100:
        return 6

    
def CutenessBuckets_2_diff_breakpoint(pawpularity_score):
    if pawpularity_score <= 75:
        return 0
    elif pawpularity_score <= 100:
        return 1

df = pd.read_csv(csv_path)
df['cuteness_bin10'] = df['Pawpularity'].apply(CutenessBuckets)
df['cuteness_bin2'] = df['Pawpularity'].apply(CutenessBuckets_2)
df['cuteness_bin3'] = df['Pawpularity'].apply(CutenessBuckets_3)
df['cuteness_bin4'] = df['Pawpularity'].apply(CutenessBuckets_4)
df['cuteness_bin5'] = df['Pawpularity'].apply(CutenessBuckets_5)
df['cuteness_bin7'] = df['Pawpularity'].apply(CutenessBuckets_7)
df['CutenessBuckets_2_diff_breakpoint'] = df['Pawpularity'].apply(CutenessBuckets_2_diff_breakpoint)

labels_raw10 = np.array(df['cuteness_bin10'])
labels_raw2 = np.array(df['cuteness_bin2'])
labels_raw3 = np.array(df['cuteness_bin3'])
labels_raw4 = np.array(df['cuteness_bin4'])
labels_raw5 = np.array(df['cuteness_bin5'])
labels_raw7 = np.array(df['cuteness_bin7'])
labels_raw2_diff = np.array(df['CutenessBuckets_2_diff_breakpoint'])

train_labels_raw10 =  labels_raw10[:7921]
dev_labels_raw10 =  labels_raw10[7920:8916]
test_labels_raw10 = labels_raw10[8916:]

train_labels_raw2 =  labels_raw2[:7921]
dev_labels_raw2 =  labels_raw2[7920:8916]
test_labels_raw2 = labels_raw2[8916:]

train_labels_raw3 =  labels_raw3[:7921]
dev_labels_raw3 =  labels_raw3[7920:8916]
test_labels_raw3 = labels_raw3[8916:]

train_labels_raw4 =  labels_raw4[:7921]
dev_labels_raw4 =  labels_raw4[7920:8916]
test_labels_raw4 = labels_raw4[8916:]

train_labels_raw5 =  labels_raw5[:7921]
dev_labels_raw5 =  labels_raw5[7920:8916]
test_labels_raw5 = labels_raw5[8916:]

train_labels_raw7 =  labels_raw7[:7921]
dev_labels_raw7 =  labels_raw7[7920:8916]
test_labels_raw7 = labels_raw7[8916:]

train_labels_raw2diff =  labels_raw2_diff[:7921]
dev_labels_raw2diff =  labels_raw2_diff[7920:8916]
test_labels_raw2diff = labels_raw2_diff[8916:]

## Models - 4 Buckets

I chose 4 buckets because we were getting somewhat better performance with larger buckets, and I think it would still be insightful to have quartiles

### GBM

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

print('-'*10, 'Gradient Boosting Classifier ', '-'*10)
clf_gb = GradientBoostingClassifier()
clf_gb.fit(train_data_temp, train_labels_raw4)
preds = clf_gb.predict(dev_data_temp)
print ('Gradient Boosting Model accuracy: %3.4f' %clf_gb.score(dev_data_temp, dev_labels_raw4))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print('-'*10, 'Random Forest Classifier ', '-'*10)
clf_rf = RandomForestClassifier()
clf_rf.fit(train_data_temp, train_labels_raw4)
preds_rf = clf_rf.predict(dev_data_temp)
print ('Random Forest Model accuracy: %3.4f' %clf_rf.score(dev_data_temp, dev_labels_raw4))

## Error Analysis

We want to understand why our performance is pretty low. An error analysis can help with this. Is or model close, or is it far off from the labels?

In [ ]:
#look at gradient boosting preds
diffs = dev_labels_raw4 - preds_gb

#mean
differences_mean = np.mean(np.absolute(diffs), dtype=np.float64)
print('Mean differences: ', differences_mean)

#get examples of most incorrect predictions
#max
result_max = np.where(differences_mean == np.amax(differences_mean))
print('List of Indices of maximum element :', result_max[0])

#min
result_min = np.where(differences_mean == np.amin(differences_mean))
print('List of Indices of minimum element :', result_min[0])

#correct
result_correct = np.where(differences_mean == 0)
print('List of Indices of correct predictions:', result_correct)